In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

In [ ]:
sys.path.insert(0,"/home/container/notebooks/missions-cways/mobivia-revenue-forecast/mobivia_revenue_forecast/src")

from get_score_params_predict import get_score_params_predict
from get_clean_feature_and_ca import get_clean_feature_and_ca
from display_curve import display_curve

# Constantes

In [ ]:
predict_year_list=[2022, 2023]
train_year_list=[2000, 2001,2002,2003,2004,2005,2006,2007,2008,2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,2017,2018, 2019, 2020,2021]
to_keep_index_list=[k for k in range(2000, 2024)]#pour l'étape de pré-traitement : toutes les lignes dont on aura besoin

growth_rate_bool=False #True pour avoir le modèle en taux de croissance
ca_col_str="NOR_CA_TOT_ACTU"

SELECTED_VARIABLES=[
    "CONSO_FINALE_MENAGES",
    "VP610",
    "IND_PRIX_UTILISATION_VEHICULE"
]

# Import et pré-traitement

In [ ]:
file_path = "/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/inputs/input_2023/Consolidation_data_all_init_2023_input_notebook.xlsx"

In [ ]:
france_pdf = pd.read_excel(open(file_path, 'rb'),sheet_name='France',skiprows=0, index_col = 0)

In [ ]:
feature_pdf, ca_pdf=get_clean_feature_and_ca(france_pdf, ca_col_str, to_keep_index_list, growth_rate_bool)

# Prédiction, score, paramètres

In [ ]:
param_dict, score, estimated_serie = get_score_params_predict(feature_pdf[SELECTED_VARIABLES], ca_pdf, [2022, 2023],True)

In [ ]:
print("score", score)
print("params_dict", param_dict)

# Sauvegarde résultats et paramètres du modèle dans "ouputs"

In [ ]:
param_df = pd.DataFrame.from_dict(param_dict, orient='index')
param_df.to_excel("/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/outputs/resultats_2023/params_2023_Norauto.xlsx")
param_df.head()

In [ ]:
compare_results_df = pd.DataFrame(
    list(zip(ca_pdf.tolist(), estimated_serie)),
    columns = ['real_values', 'estimated_values'],
    index = ca_pdf.index
)

compare_results_df.to_excel("/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/outputs/resultats_2023/prev_2023_Norauto.xlsx")
compare_results_df.head(n=50)

# Quelques graphs

In [ ]:
display_curve(compare_results_df, train_year_list, growth_rate_bool)
#résidus négatifs : estimated_value supérieure à real_value
#exprimés en % d'écart => fonctionne mal avec les taux de croissances
#pour le taux de croissance, il faudrait juste une différence, pas relative

In [ ]:
plt.figure(figsize=(12,8))
#compare_results_df.drop([2021, 2022], axis=0).plot()
compare_results_df.plot()
plt.xlabel("Année")
plt.ylabel("Chiffre d'affaires")
plt.show()